In [4]:
#ignore
#Import the Zillow Data

import requests
import os

url = "https://zillow-com1.p.rapidapi.com/property"

querystring = {"address":"2554 Tennyson Ave, Strathmoor Village, KY 40205"}

X_RAPIDAPI_KEY = os.environ.get('RAPIDAPI_KEY')

headers = {
	"X-RapidAPI-Key": X_RAPIDAPI_KEY,
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'listingProvider': None, 'zpid': 73517281, 'buildingPermits': None, 'propertyTaxRate': 0.84, 'contact_recipients': [{'agent_reason': 1, 'zpro': None, 'recent_sales': 0, 'review_count': 0, 'display_name': 'Jennifer Hicks', 'zuid': 'X1-ZUrt2h06wti4nd_7aoc0', 'rating_average': 0, 'badge_type': 'Premier Agent', 'phone': {'prefix': '317', 'areacode': '502', 'number': '2843'}, 'image_url': 'https://photos.zillowstatic.com/h_n/ISfko4gu6ud2no0000000000.jpg'}], 'zipcode': '40205', 'openHouseSchedule': {}, 'longitude': -85.6795, 'zestimateLowPercent': None, 'address': {'community': None, 'city': 'Strathmoor Village', 'state': 'KY', 'neighborhood': None, 'subdivision': None, 'streetAddress': '2554 Tennyson Ave', 'zipcode': '40205'}, 'cityId': 12455, 'timeOnZillow': '10 days', 'url': '/homedetails/2554-Tennyson-Ave-Strathmoor-Village-KY-40205/73517281_zpid/', 'zestimate': None, 'imgSrc': 'https://photos.zillowstatic.com/fp/7a17b04fd1ba5b13a122c4a369d7da73-p_d.jpg', 'description': 'Beautiful Home 

In [2]:
import requests
import os

def get_zillow_property_data(address):
    url = "https://zillow-com1.p.rapidapi.com/property"
    headers = {
        "X-RapidAPI-Key": os.environ.get('RAPIDAPI_KEY'),
        "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
    }
    querystring = {"address": address}

    try:
        response = requests.get(url, headers=headers, params=querystring)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print("API request failed with status code:", response.status_code)
            return None

    except Exception as e:
        print("An error occurred:", str(e))
        return None

# Example usage:
address = "2554 Tennyson Ave, Strathmoor Village, KY 40205"
property_data = get_zillow_property_data(address)

if property_data:
    print(property_data)
else:
    print("No data found for the provided address.")


{'listingProvider': None, 'zpid': 73517281, 'buildingPermits': None, 'propertyTaxRate': 0.84, 'contact_recipients': [{'agent_reason': 1, 'zpro': None, 'recent_sales': 0, 'review_count': 63, 'display_name': 'Jackie Wilson', 'zuid': 'X1-ZUvadhbknkatqh_7rj5o', 'rating_average': 5, 'badge_type': 'Premier Agent', 'phone': {'prefix': '206', 'areacode': '502', 'number': '1377'}, 'image_url': 'https://photos.zillowstatic.com/h_n/ISmmz8var45u8u1000000000.jpg'}], 'zipcode': '40205', 'openHouseSchedule': {}, 'longitude': -85.6795, 'zestimateLowPercent': None, 'address': {'community': None, 'city': 'Strathmoor Village', 'state': 'KY', 'neighborhood': None, 'subdivision': None, 'streetAddress': '2554 Tennyson Ave', 'zipcode': '40205'}, 'cityId': 12455, 'timeOnZillow': '20 days', 'url': '/homedetails/2554-Tennyson-Ave-Strathmoor-Village-KY-40205/73517281_zpid/', 'zestimate': None, 'imgSrc': 'https://photos.zillowstatic.com/fp/7a17b04fd1ba5b13a122c4a369d7da73-p_d.jpg', 'description': 'Beautiful Home 

LOJIC Data: 

In [7]:
import requests
import pandas as pd

def get_address_info(address):
    api_url = "https://gis.lojic.org/maps/rest/services/LojicSolutions/OpenDataAddresses/MapServer/0/query"
    
    # Define the query parameters to search by FULL_ADDRESS
    params = {
        "where": f"FULL_ADDRESS = '{address}'",
        "outFields": "ZONING_CODE,PARCELID,HISTDIST,MUNI_NAME,FIREDIST,COUNTY",
        "f": "json"
    }
    
    try:
        response = requests.get(api_url, params=params)
        if response.status_code == 200:
            data = response.json()
            features = data.get("features", [])
            if features:
                # Create a DataFrame from the response data
                df = pd.DataFrame([feature["attributes"] for feature in features])
                return df
            else:
                return "Address not found"
        else:
            return "API request failed"
    except Exception as e:
        return str(e)

# Example usage
input_address = "2554 Tennyson Ave, Strathmoor Village, KY 40205" 
address_info = get_address_info(input_address)


if isinstance(address_info, pd.DataFrame):
    print("Address Information:")
    print(address_info)
else:
    print(f"No information found for the address")


No information found for the address


In [14]:
from flask import Flask, request, jsonify
import requests

app = Flask(__name__)

# Define the GeoJSON API endpoint
geojson_url = "https://gis.lojic.org/maps/rest/services/LojicSolutions/OpenDataAddresses/MapServer/0/query?outFields=*&where=1%3D1&f=geojson"

def get_address_data(address):
    try:
        response = requests.get(geojson_url)
        response.raise_for_status()

        geojson_data = response.json()
        features = geojson_data.get("features", [])

        if not features:
            return "No data found in the GeoJSON."

        for feature in features:
            attributes = feature.get("properties", {})
            full_address = attributes.get("FULL_ADDRESS", "")
            
            if address.lower() in full_address.lower():
                return attributes

        return "Address not found in the GeoJSON."
    except requests.exceptions.RequestException as e:
        return f"GeoJSON request failed: {str(e)}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

@app.route('/get_address_info', methods=['GET'])
def get_address_info():
    address = request.args.get('address', default='', type=str)
    if address:
        data = get_address_data(address)
        if isinstance(data, dict):
            return jsonify(data)
        else:
            return data, 404
    else:
        return "Please provide a valid address as a query parameter.", 400

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
/Users/rileynoe/.pyenv/versions/3.9.5/lib/python3.9/site-packages/traitlets/traitlets.py:2412: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use 'hmac-sha256' instead of '"hmac-sha256"' if you require traitlets >=5.
  warn(
/Users/rileynoe/.pyenv/versions/3.9.5/lib/python3.9/site-packages/traitlets/traitlets.py:2366: FutureWarning: Supporting extra quotes around Bytes is deprecated in traitlets 5.0. Use '145c41d2-22ac-47bb-9a5f-b8f6ed5647cb' instead of 'b"145c41d2-22ac-47bb-9a5f-b8f6ed5647cb"'.
  warn(
Traceback (most recent call last):
  File "/Users/rileynoe/.pyenv/versions/3.9.5/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/rileynoe/.pyenv/versions/3.9.5/lib/python3.9/site-packages/traitlets/config/application.py", line 984, in launch_instance
    app.initialize(argv)
  File "/Users/ri

SystemExit: 1

/Users/rileynoe/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
